<a href="https://colab.research.google.com/github/SucharitaGorai/Registration_form/blob/main/data_collection/hugging_face_text_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datasets import load_dataset
from tqdm.auto import tqdm
import json
import re
import sign_language_translator as slt
lang = slt.languages.text.Urdu()
symbols = "".join(lang.SYMBOLS)

## Download `Urdu`

In [ ]:
ur_dates = ["20230901", "20230920", "20231001", "20231020", "20231101", "20231120", "20231201", "20231220", "20240101", "20240120"]

In [ ]:
for d in ur_dates[:1]:
    print(d)
    load_dataset("wikipedia", language="ur", date=d, beam_runner="DirectRunner", trust_remote_code=True)
load_dataset("wikimedia/wikipedia", "20231101.ur")  # 1.2M unique words

## Download `English`

In [ ]:
# English
load_dataset("wikitext", 'wikitext-103-v1')
load_dataset("wikitext", 'wikitext-2-v1')
load_dataset("bookcorpus")
load_dataset("wikimedia/wikisource", "20231201.en")

## Download `Hindi`

In [ ]:
hi_dates = ['20230901', '20230920', '20231001', '20231020', '20231101', '20231120', '20231201', '20231220', '20240101', '20240120']

In [ ]:
for d in hi_dates:
    print(d)
    load_dataset("wikipedia", language="hi", date=d, beam_runner="DirectRunner", trust_remote_code=True)
load_dataset("wikimedia/wikipedia", "20231101.hi")
load_dataset("wikimedia/wikisource", "20231201.hi")

## `Save`

In [ ]:
lines = {}
words = {}

In [ ]:
dataset = load_dataset("wikimedia/wikipedia", "20231101.ur")
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 200154
    })
})

In [ ]:
for row in (bar:=tqdm(dataset["train"])):
    for l in row["text"].splitlines():
        l = l.strip()
        lines[l] = lines.get(l, 0) + 1
    for l in row["title"].splitlines():
        l = l.strip()
        lines[l] = lines.get(l, 0) + 1
len(lines), sum(lines.values())

  0%|          | 0/200154 [00:00<?, ?it/s]

(1340954, 4159209)

In [ ]:
for d in ur_dates[:]:
    print(d, end="\t")
    dataset = load_dataset("wikipedia", language="ur", date=d, beam_runner="DirectRunner", trust_remote_code=True)
    for row in dataset["train"]:
        for l in row["text"].splitlines():  #! new line can mark end/start of a sentence
            l = l.strip()
            lines[l] = lines.get(l, 0) + 1
        for l in row["title"].splitlines():
            l = l.strip()
            lines[l] = lines.get(l, 0) + 1
    print(len(lines), sum(lines.values()))

20230901	1347354 8259897
20230920	1347517 12373826
20231001	1347631 16498572
20231020	1347765 20647286
20231101	1347765 24806563
20231120	1357873 28987707
20231201	1363371 33180229
20231220	1371244 37384207
20240101	1380331 41606119
20240120	1517672 45841011


In [ ]:
sorted_lines = (sorted(lines.items(), key=lambda x: x[1], reverse=True))
len(sorted_lines), sorted_lines[100:200], sorted_lines[-1000:-900]

In [ ]:
import json
base="/Users/mudassar.iqbal/Library/CloudStorage/GoogleDrive-mdsriqb@gmail.com/My Drive/sign-language-translator/sign-language-datasets/temp/temp/raw_corpora"
with open(base+"/ur_wikipedia_lines.json", 'w', encoding="utf-8")as f:
    json.dump(dict(sorted_lines), f, ensure_ascii=False, indent=0)
    # json.dump(dict([l for l in sorted_lines if l[1]>50]), f, ensure_ascii=False, indent=0)